PRACTICE. Сбор и обработка данных

Важно иметь в виду: все задания необходимо выполнять с использованием данных пользователей, зарегистрировавшихся в 2018 году.

In [1]:
import pandas as pd
import numpy as np

purchase_df = pd.read_csv(r'C:\Users\Yana\Documents\purchase.csv', sep=',')
events_df = pd.read_csv(r'C:\Users\Yana\Documents\7_4_Events.csv', sep=',')

In [2]:
purchase_df.info()
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


In [3]:
events_df['selected_level'].value_counts()

medium    17408
easy       9011
hard       4667
Name: selected_level, dtype: int64

In [4]:
#events_df.info()
# events_df['start_time'] = pd.to_datetime(events_df['start_time']) --> ERROR, have to separate date on T

In [5]:
s = '2016-05-11T23:40:55'
lst = s.split('T')
lst

['2016-05-11', '23:40:55']

In [6]:
# Для начала выведем первые 10 строк датафрейма events_df с помощью команды head(), чтобы ознакомиться с данными.
events_df.head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
5,28908,tutorial_start,NaN,2016-05-12T02:11:29,17563.0,12584
6,28909,tutorial_finish,NaN,2016-05-12T02:14:14,17563.0,12584
7,28910,level_choice,hard,2016-05-12T05:30:25,NaN,12584
8,28911,pack_choice,NaN,2016-05-12T05:38:24,NaN,12584
9,28912,tutorial_start,NaN,2016-05-12T07:19:01,17564.0,12583


In [7]:
# Мы, конечно, можем уже что-то сказать по поводу данных, посмотрев на первые 10 строк, 
# но лучше использовать более точные методы анализа. 
# Воспользуемся методом info(), который позволяет получить общую информацию о датафрейме, 
# для того чтобы оценить, какие данные содержатся в датафрейме events_df.
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


Итак, мы видим, что датафрейм events_df содержит 35808 строк и шесть столбцов. При этом все строки содержат значения в столбцах id, user_id, start_time, event_type, а остальные столбцы содержат пропущенные значения (например, столбец selected_level содержит непустые значения только в 4412 строках).

To retrieve all the rows which startwith required string

dataFrameOut = dataFrame[dataFrame['column name'].str.match('string')]

To retrieve all the rows which contains required string

dataFrameOut = dataFrame[dataFrame['column name'].str.contains('string')]

In [8]:
# events_df['start_time'] = pd.to_datetime(events_df['start_time']) 
# ERROR !!!

In [9]:
# Сколько строк содержится в датафрейме events_df? 66959
events_2018 = events_df[events_df['start_time'].str.contains('2018')].copy()
events_2018['start_time'] = pd.to_datetime(events_2018['start_time'])
events_2018.shape
# events_2018.head()

(66952, 6)

In [10]:
# Сколько непустых значений содержится в столбце selected_level? 66959 - 58612 = 8347 #8342
# print(events_2018['selected_level'].isnull().sum()) 58612
events_2018.info() #8340

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66952 entries, 51405 to 118356
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66952 non-null  int64         
 1   event_type      66952 non-null  object        
 2   selected_level  8340 non-null   object        
 3   start_time      66952 non-null  datetime64[ns]
 4   tutorial_id     32950 non-null  float64       
 5   user_id         66952 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


In [11]:
# Исследуем пропуски
# Наличие пропущенных строк обусловлено тем, что не все из параметров selected_level и tutorial_id 
# присутствуют в каждом из событий. Эти параметры заполнятся в зависимости от event_type.
# К примеру, посмотрим на events_df, оставив в нём только такие строки, где event_type = level_choice.

events_2018[events_2018['event_type'] == 'level_choice'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8340 entries, 51424 to 118353
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8340 non-null   int64         
 1   event_type      8340 non-null   object        
 2   selected_level  8340 non-null   object        
 3   start_time      8340 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         8340 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.1+ KB


In [12]:
# Как видно, этот срез датафрейма не содержит пропущенных значений в столбце selected_level,
# но зато содержит пропуски в tutorial_id. 
# Это связано с тем, что для событий типа level_choice не предусмотрена запись параметра tutorial_id.
# Теперь проверим аналогичные данные, но при условии, что срез будет содержать данные о 
# событиях tutorial_start и tutorial_finish.
events_2018[events_2018['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32950 entries, 51412 to 118351
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              32950 non-null  int64         
 1   event_type      32950 non-null  object        
 2   selected_level  0 non-null      object        
 3   start_time      32950 non-null  datetime64[ns]
 4   tutorial_id     32950 non-null  float64       
 5   user_id         32950 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.8+ MB


Столбец selected_level в таком срезе не содержит ни одного значения. Причина этого в том, что для событий, связанных с обучением, отсутствуют параметры выбора уровня сложности selected_level. А вот остальные столбцы заполнены полностью.

In [13]:
# Вызовем метод describe(), чтобы оценить характеристики каждого столбца. 
# По умолчанию метод describe() выдаёт характеристики только по столбцам с численными типами 
# (например, int64, float64). Мы вызовем этот метод с параметром include='all' для того,
# чтобы отображать характеристики для всех столбцов.
events_2018.describe(include='all')

C:\Users\Yana\AppData\Local\Temp/ipykernel_10388/1180975805.py:5: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  events_2018.describe(include='all')


,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66952.000000,66952,8340,66952,32950.000000,66952.000000
unique,NaN,5,3,66802,NaN,NaN
top,NaN,registration,medium,2018-03-13 09:37:43,NaN,NaN
freq,NaN,19926,4644,9,NaN,NaN
first,NaN,NaN,NaN,2018-01-01 03:48:40,NaN,NaN
last,NaN,NaN,NaN,2018-12-31 23:18:06,NaN,NaN
mean,113783.500000,NaN,NaN,NaN,40531.839332,37780.500553
std,19327.521948,NaN,NaN,NaN,5212.855551,5750.894235
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97045.750000,NaN,NaN,NaN,36008.000000,32848.750000


In [14]:
# Давайте оценим, какие уникальные события есть в колонках event_type и selected_level.
events_2018['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

In [15]:
events_2018['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [16]:
# Также оценим, какое количество пользователей совершали события:
events_df['user_id'].nunique()

74908

In [17]:
# Теперь нам нужно проверить, как события, совершённые пользователями, связаны с последующими оплатами пакетов вопросов.
# Выведем первые 10 строк датафрейма purchase_df с помощью команды head().
purchase_df.head(10)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12 10:34:16,100
1,15675,12985,2016-05-13 08:25:56,50
2,15676,12828,2016-05-13 16:33:46,50
3,15677,12598,2016-05-14 01:09:37,150
4,15678,13037,2016-05-14 01:24:46,100
5,15679,12989,2016-05-14 02:19:20,200
6,15680,12684,2016-05-14 03:10:14,100
7,15681,12656,2016-05-14 03:42:48,100
8,15682,12624,2016-05-14 06:00:45,50
9,15683,13136,2016-05-14 07:31:03,50


In [18]:
# Воспользуемся методом info(), который позволяет получить общую информацию о датафрейме,
# для того чтобы оценить, какие данные содержатся в датафрейме purchase_df.
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              5956 non-null   int64         
 1   user_id         5956 non-null   int64         
 2   event_datetime  5956 non-null   datetime64[ns]
 3   amount          5956 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 186.2 KB


Важно! Напоминаем, что все задания необходимо выполнять с использованием данных пользователей, зарегистрировавшихся в 2018 году.

In [19]:
# Сколько строк содержится в датафрейме purchase_df?
purchase_df['year'] = purchase_df['event_datetime'].dt.year
purchase_2018 = purchase_df[purchase_df['year'] == 2018].copy()
purchase_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1602 entries, 1164 to 2765
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1602 non-null   int64         
 1   user_id         1602 non-null   int64         
 2   event_datetime  1602 non-null   datetime64[ns]
 3   amount          1602 non-null   int64         
 4   year            1602 non-null   int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 75.1 KB


In [20]:
# Снова обратимся к методу describe(), чтобы оценить характеристики каждого столбца датафрейма purchase_df.
purchase_2018.describe()

,id,user_id,amount,year
count,1602.000000,1602.000000,1602.000000,1602.0
mean,17638.500000,37665.920724,110.705368,2018.0
std,462.601881,5829.659702,54.713226,0.0
min,16838.000000,27594.000000,25.000000,2018.0
25%,17238.250000,32719.750000,50.000000,2018.0
50%,17638.500000,37495.500000,100.000000,2018.0
75%,18038.750000,42978.250000,150.000000,2018.0
max,18439.000000,47712.000000,300.000000,2018.0
